# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [1]:
import pandas as pd

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
from pathlib import Path

current = Path('.').resolve()
rev = current.parents[0] / 'лаба 2/data/reviews_sample.csv'
rec = current.parents[0] / 'лаба 5/data/recipes_sample.csv'

recipes,reviews = pd.read_csv(rec,sep=',',header=0,engine='python')[['id', 'name', 'minutes', 'submitted', 'description','n_ingredients']],pd.read_csv(rev,sep=',',header=0,engine='python')


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [3]:
rev_perc,rec_perc= reviews.sample(frac=0.05),recipes.sample(frac=0.05)

with pd.ExcelWriter("data/7_1_2_result.xlsx") as writer:
    rec_perc.to_excel(writer,sheet_name="Рецепты")
    rev_perc.to_excel(writer,sheet_name="Отзывы")

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [4]:
import xlwings as xw

wb = xw.Book('data/7_1_2_result.xlsx')

sheet = wb.sheets['Рецепты']
sheet["H1"].value = 'seconds_assign'
sheet["H2:H1501"].options(index=False,header=False).value = rec_perc['minutes']*60



4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [5]:
sheet["I1"].value = 'seconds_formula'
range_name = xw.Range('D2').expand('down').address
sheet["I2:I1501"].formula = f'={range_name}*60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [6]:
xw.Range('H1:I1').api.Font.Bold=True
xw.Range('H1:I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [7]:
def color(rng):
    for _,v in enumerate(rng):
        if v.value < 5:
            v.color = (0,255,0)
        elif v.value>10:
            v.color = (255,0,0)
        else:
            v.color=(255,255,0)

color(xw.Range('D2').expand('down'))

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [8]:
xw.Range('J1').api.Font.Bold=True
xw.Range('J1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
xw.Range('J1').value = 'n_reviews'

addr = wb.sheets['Отзывы']['D2'].expand('down').get_address(include_sheetname=True)
rng = xw.Range('B2').get_address(row_absolute=False)
sheet['J2'].expand('down').formula = f'=COUNTIF({addr},{rng})'

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [9]:
def validate(rng):
    lst = sheet['B2'].expand('down').value
    for row in rng.rows:
        if row[5].value<0 or row[5].value>5 or row[3].value not in lst:
            row.color = (255,0,0)
        

sheet2 = wb.sheets['Отзывы']
rng = sheet2['A2'].expand('right').expand('down')

validate(rng)

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
import csv
import numpy as np
arr = np.array([])
with open('data/recipes_model.csv','r') as file:
    readdr = csv.reader(file,delimiter = ' ')
    arr = np.array(list(map(list,readdr)),dtype=object)
# print(readdr)
    
book_pth = current / 'data/recipes_model.xlsx'

with open(book_pth,'w') as file:
    pass

wb2 = xw.Book(book_pth)
# models = wb2.sheets['Модель']
models = wb2.sheets.add('Модель')

models['A2'].options(np.array).value = arr


10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 